<center><h1>Datos Meteorológicos con Métodos de Interpolación</h1></center>  
<center><img src="clima.png" width=270 height=270></center>  

### **Sección: 305C1**
### **Estudiantes:**
Manuel Nava 30.822.007  
Juan Wu 30.391.117  
José Farrauto 30.696.288  

---

## Interpolación de Taylor

---

## Interpolación de Lagrange

---

## Interpolación de Hermite

---

## Interpolación Polinómicas a Trozos

<center><h2>Gracias!!</h2></center>